# DXF Processing

The purpose of this notebook is to explore dxf processing options available in python. Ultimately we'll create a method to extract polylines from a dxf file and a csv file from them to import into Aegis to automatically create rings.


## Date

2019-07-25
2019-07-24


## TODO:
- Split the notebook up into basics of dxf and another one on processing the dxf files into Aegis files


# Code References/Boilerplate

The only thing we need for this ability to turn off the autoscroll, which the bit of javascript does

In [20]:
%%javascript
//Disable autoscroll in the output cells
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

# Exploring ezdxf

repo: https://github.com/mozman/ezdxf


## ezdxf


### Abstract


A Python package to create and modify DXF drawings, independent from the DXF
version. You can open/save every DXF file without losing any content (except comments),
Unknown tags in the DXF file will be ignored but preserved for saving. With this behavior
it is possible to open also DXF drawings that contains data from 3rd party applications.

### Quick-Info

- ezdxf is a Python package to create new DXF files and read/modify/write existing DXF files
- the intended audience are developers
- requires at least CPython 3.5, for Python 2 support use ezdxf < 0.9
- OS independent
- additional required packages: [pyparsing](https://pypi.org/project/pyparsing/)
- MIT-License
- read/write/new support for DXF versions: R12, R2000, R2004, R2007, R2010, R2013 and R2018
- additional read support for DXF versions R13/R14 (upgraded to R2000)
- additional read support for older DXF versions than R12 (upgraded to R12)
- preserves third-party DXF content
- additional fast DXF R12 writer, that creates just an ENTITIES section with support for the basic DXF entities

a simple example:

```python
import ezdxf

drawing = ezdxf.new(dxfversion='R2010')  
# alternative:  use the AutoCAD release name 
# ezdxf.new(dxfversion='R2010')
modelspace = drawing.modelspace()
modelspace.add_line((0, 0), (10, 0), dxfattribs={'color': 7})
drawing.layers.new('TEXTLAYER', dxfattribs={'color': 2})

# use set_pos() for proper TEXT alignment:
# the relations between halign, valign, insert and align_point are tricky.
modelspace.add_text(
    'Test', 
    dxfattribs={
        'layer': 'TEXTLAYER'
    }).set_pos((0, 0.2), align='CENTER')
    
drawing.saveas('test.dxf')
```

example for the *r12writer*, writes a simple DXF R12 file without in-memory structures:

```python
from random import random
from ezdxf.r12writer import r12writer

MAX_X_COORD = 1000.0
MAX_Y_COORD = 1000.0
CIRCLE_COUNT = 100000

with r12writer("many_circles.dxf") as dxf:
    for i in range(CIRCLE_COUNT):
        dxf.add_circle((MAX_X_COORD*random(), MAX_Y_COORD*random()), radius=2)
```

The r12writer supports only the ENTITIES section of a DXF R12 drawing, no HEADER, TABLES or BLOCKS section is
present, except FIXED-TABLES are written, than some additional predefined text styles and line types are available.

### Installation

Install with pip:

    pip install ezdxf

Install develop version (only if you have to)::

    pip install git+https://github.com/mozman/ezdxf.git@develop

For Python 2 users:

    pip install ezdxf<0.9


or from source:

    python setup.py install

### Website


https://ezdxf.mozman.at/

### Documentation

Documentation of development version at https://ezdxf.mozman.at/docs

Documentation of latest release at http://ezdxf.readthedocs.io/

### Contribution


The source code of ezdxf can be found at GitHub.com:

http://github.com/mozman/ezdxf.git

Only pull requests for the **develop** branch will be accepted.

### Feedback


Issue Tracker at:

http://github.com/mozman/ezdxf/issues

Questions and Feedback at Google Groups:

https://groups.google.com/d/forum/python-ezdxf

python-ezdxf@googlegroups.com

Feedback is greatly appreciated.

Manfred



In [21]:
# Import Standard Modules
import os
from pathlib import Path
import ezdxf

import numpy as np

#-------------
# decide whether charts will be displayed interactively or in a format that exports to pdf

# Interactive -----------------
# For interactive notebook uncomment:
%matplotlib notebook

# PDF -----------------
# For pdf plotting uncomment:
# %matplotlib inline
# import warnings
# warnings.filterwarnings('ignore')

# ------------------
# matplotlib
from mpl_toolkits.mplot3d.axes3d import Axes3D
import matplotlib.pyplot as plt 
import matplotlib as mpl

# uncomment the following if you have more then 40 plots
# mpl.rcParams['figure.max_open_warning'] = 60

mpl.rc('font',family='monospace') # all font on the plot will be monospace

# get plotted images to be stored as png and pdf so that the most appropriate form can be selected
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

#--------------
# Import from Custom Packages
# load any of the common modules that are required
from geometry.vector import Vector3
from geometry.vector import Point3
from geometry.plane import Plane3

from plots import standard_figure 

from units.units import Units
# initialize libraries
u = Units()

In [22]:
dxf_folders = ('95 3W MP2A LHS1E EXT', 
               '95 3W MP3 LHS1E', 
               '100 4W CUT4 LH07MB', 
               '100 4W CUT4 LH07W')

working_dir = Path(os.getcwd())

# create the path to the packages folder
dxf_folders = tuple(working_dir.joinpath(f) for f in dxf_folders)

In [23]:
# find the dxf files in the first folder

dxf_files = [x for x in dxf_folders[0].glob('*.dxf')]

# for f in dxf_files:
#     print(f)

In [24]:
print(dxf_files[0])
dwg = ezdxf.readfile(dxf_files[0])

# get a reference to model space
msp = dwg.modelspace()

C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\95 3W MP2A LHS1E EXT\95 3W MP2A LH1E_EXTENSION.dxf


In [25]:
# extract all of the polylines and add them to the dataset
dataset = []
for e in msp.query('POLYLINE'):    
    x, y, z = zip(*e.points())
    line = {}
    line['x'] = x
    line['y'] = y
    line['z'] = z
    dataset.append(line)

In [26]:
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})

for ds in dataset:
    ax.plot(ds["x"], ds["y"], ds["z"])
    
fig.show()

<IPython.core.display.Javascript object>

###

In [27]:
# find the dxf files in the first folder
for f in dxf_folders:
    dxf_files = [x for x in f.glob('*.dxf')]
       
    dataset = []
    for f in dxf_files:
        print(f)
        dwg = ezdxf.readfile(f)
        
        # get a reference to model space
        msp = dwg.modelspace()

        # extract all of the polylines and add them to the dataset                
        for e in msp.query('POLYLINE'):            
            x, y, z = zip(*e.points())
            line = {}
            line['x'] = x
            line['y'] = y
            line['z'] = z
            dataset.append(line)

            
    # At this point we can create the csv file based on what we have here
    # will need to pick a hole and generate a ring plane and test to see how far the collar is from it and go from there
    
    
    # plot the holes
    fig, ax = plt.subplots(subplot_kw={'projection': '3d'})

    for ds in dataset:
        ax.plot(ds["x"], ds["y"], ds["z"])

    fig.show()

C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\95 3W MP2A LHS1E EXT\95 3W MP2A LH1E_EXTENSION.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\95 3W MP2A LHS1E EXT\95 3W MP2A LH1E_EXTENSION_SLIPE.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\95 3W MP2A LHS1E EXT\95 3W MP2A LH1E_r8-r17.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\95 3W MP2A LHS1E EXT\95 3W MP2A LH1E_Rn1-Rn4.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\95 3W MP2A LHS1E EXT\95 3W MP2A LH1E_S1-R7.dxf


<IPython.core.display.Javascript object>

C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\95 3W MP3 LHS1E\95 3W MP2a LHSS 02S_blast1.dxf


<IPython.core.display.Javascript object>

C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\100 4W CUT4 LH07MB\100 4W Cut4 LHPS 7EMB slot2 rings.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\100 4W CUT4 LH07MB\100 4W Cut4 LHPS 7EMB slot3 rings.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\100 4W CUT4 LH07MB\100 4W Cut4 LHPS 7EMBr3-r7 rings.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\100 4W CUT4 LH07MB\100 4W Cut4 LHPS 7EMBr8-r11 rings.dxf


<IPython.core.display.Javascript object>

C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\100 4W CUT4 LH07W\100 4W CUT3 LHP 07W blast 2 _R1a-R10.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\100 4W CUT4 LH07W\100 4W CUT3 LHP 07W blast 2 _R1a-R3.dxf
C:\personal\repositories\git\jupyter\projects\holes to Aegis delimited format\100 4W CUT4 LH07W\100 4W CUT3 LHP 07W SLOT.dxf


<IPython.core.display.Javascript object>

# Polylines - Group by Layer

- Group the lines together by layer in a dictionary, then plot them

In [28]:
# find the dxf files in the first folder
for f in dxf_folders:
    dxf_files = [x for x in f.glob('*.dxf')]
       
    layers = {}
    for f in dxf_files:                        
        dwg = ezdxf.readfile(f)
        
        # get a reference to model space
        msp = dwg.modelspace()

        # extract all of the polylines and add them to the dataset                
        for e in msp.query('POLYLINE'):            
            layers.setdefault(e.dxf.layer, []).append(e)

# for k in layers.keys():
#     print(k)

In [29]:
dataset = []
for e in layers['RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R5']:            
        x, y, z = zip(*e.points())
        line = {}
        line['x'] = x
        line['y'] = y
        line['z'] = z
        dataset.append(line)
            
# plot the holes
fig, ax = plt.subplots(subplot_kw={'projection': '3d'})

for ds in dataset:
    ax.plot(ds["x"], ds["y"], ds["z"])

ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')    
    
fig.show()

<IPython.core.display.Javascript object>

# Plot layer-by-layer and fit plane

In [30]:
from best_fit_plane import best_fit_plane

from geometry.vector import Vector3
from geometry.vector import Point3
from geometry.plane import Plane3

In [31]:
# this appears to be the problematic dxf file where it has trouble fitting the holes to a plane
f = Path("95 3W MP2A LHS1E EXT/95 3W MP2A LH1E_r8-r17.dxf")

# this appears to be how the rings are grouped within that dxf file
set1 = [
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R8A",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R8B",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R8",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R9",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R10",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R11",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R012",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R013",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R014",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R015",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R016",
    "95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R017",
]

# design 2
set2 = [
    "100 4W CUT3 LHP 07W BLAST 2 _R1A-R3_DXF_RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R4",
    "100 4W CUT3 LHP 07W BLAST 2 _R1A-R3_DXF_RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R3",
    "100 4W CUT3 LHP 07W BLAST 2 _R1A-R3_DXF_RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R2",
    "100 4W CUT3 LHP 07W BLAST 2 _R1A-R3_DXF_RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R1",
    "100 4W CUT3 LHP 07W BLAST 2 _R1A-R3_DXF_RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R1A",
]

set2_slot = [
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S1S",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S2S",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S3S",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S4S",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S5S",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S1",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S2",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S3",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S4",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S5",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S6",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S7",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S1N",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S2N",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S3N",
    "RINGDESIGN_100 04W CUT03 LHP 07W SLOT_SURVEYEXPORT_S4N",
]


set2_fanned = [
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R10",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R9",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R8",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R7",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R6",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R5",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R4",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R3",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R2",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R1",
    "RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R1A",
]


def filter_layer(
    layer, use_set1=False, use_set2=False, use_set2_slot=False, use_set2_fanned=False
):
    """
    A simple method that will return true if we are ignoring the passed in layer
    """

    if use_set1 and layer in set1:
        return True

    if use_set2 and layer in set2:
        return True

    if use_set2_slot and layer in set2_slot:
        return True

    if use_set2_fanned and layer in set2_fanned:
        return True

    return False

In [32]:
# read the dxf and extract all of the polylines and group them by layer
dwg = ezdxf.readfile(f.resolve())

# get a reference to model space
msp = dwg.modelspace()

layers = {}
# extract all of the polylines and add them to the dataset                
for e in msp.query('POLYLINE'):            
    layers.setdefault(e.dxf.layer, []).append(e)

In [33]:
# uncomment to get one plot per layer
for k, lines in layers.items():  
    
    # for the plane work
    xs = []
    ys = []
    zs = []
    
    dataset = [] # for plotting the lines
    
    for e in lines:
        x, y, z = zip(*e.points())
        xs.extend(x)
        ys.extend(y)
        zs.extend(z)
        
        line = {}
        line['x'] = x
        line['y'] = y
        line['z'] = z
        dataset.append(line)

    X = np.array(xs)
    Y = np.array(ys)
    Z = np.array(zs)
    
    print(k) 
    # ===========
    centroid = Point3(np.mean(X), np.mean(Y),np.mean(Z))
    print('Centroid = ({:.4f},{:.4f},{:.4f})'.format(*centroid))
        
    a, b, c, d, plane = best_fit_plane(X, Y, Z)

    # we can't use the d, from the best fit situation, we have to calculate it 
    # relative to the centroid of our data
    d = -centroid.dot(np.array([a, b, c]))
    normal = Vector3(a,b,c)
    print('Plane Normal = ({:.4f},{:.4f},{:.4f})'.format(*normal))
        
    # Aegis requires the azimuth of the ring plane not the normal so subtract
    # 90 degrees to it
    azimuth = np.degrees(np.arccos(normal.dot(Vector3.north_vector())))
    azimuth = np.mod(azimuth - 90.0, 360.0)
    print('Azimuth = {:.4f}'.format(azimuth))
    
    dump = np.degrees(np.arccos(normal.dot(Vector3.up_vector())))
    print('Dump = {:.4f}'.format(dump))    
    # ===========
    
    # ===========
    # setup the grid used to display the best fit plane
    
    data = np.c_[X,Y,Z]
    
    # regular grid covering the domain of the data
    mn = np.min(data, axis=0)
    mx = np.max(data, axis=0)

    if plane == 'XY':
        XX, YY = np.meshgrid(np.linspace(mn[0], mx[0], 10),     
                             np.linspace(mn[1], mx[1], 10))

        ZZ = (-a*XX - b*YY - d)/c

    elif plane == 'XZ':
        XX, ZZ = np.meshgrid(np.linspace(mn[0], mx[0], 10), 
                             np.linspace(mn[2], mx[2], 10))

        YY = (-a*XX - c*ZZ - d)/b

    elif plane == 'YZ':
        YY, ZZ = np.meshgrid(np.linspace(mn[1], mx[1], 10), 
                             np.linspace(mn[2], mx[2], 10))

        XX = (-b*YY - c*ZZ - d)/a
    #==========
    
    fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=(8, 6))
        
    for ds in dataset:
        ax.plot(ds["x"], ds["y"], ds["z"])
       
    ax.plot_wireframe(XX,YY,ZZ, color='k')
    
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')    

    fig.show()       

95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT
Centroid = (7107.8833,-23230.4002,-994.7378)
Plane Normal = (-0.0405,0.2883,0.9567)
Azimuth = 343.2455
Dump = 16.9237


<IPython.core.display.Javascript object>

95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R8A
Centroid = (7111.6793,-23224.1265,-993.0610)
Plane Normal = (-0.9455,0.3256,-0.0000)
Azimuth = 341.0000
Dump = 90.0000


<IPython.core.display.Javascript object>

95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R8B
Centroid = (7110.3287,-23223.5054,-993.0939)
Plane Normal = (-0.9925,0.1219,-0.0000)
Azimuth = 353.0000
Dump = 90.0000


...

<IPython.core.display.Javascript object>

95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R016
Centroid = (7083.4135,-23230.1613,-994.0731)
Plane Normal = (-0.9659,0.2588,-0.0000)
Azimuth = 345.0000
Dump = 90.0000


<IPython.core.display.Javascript object>

95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R017
Centroid = (7080.8172,-23230.3403,-994.6309)
Plane Normal = (-0.9659,0.2588,0.0000)
Azimuth = 345.0000
Dump = 90.0000


<IPython.core.display.Javascript object>

It seems to fit the rings nicely and generate the proper planes.

In [34]:
# plot all layers on one chart

# list of layers that seem to be part of a second design...


fig, ax = plt.subplots(subplot_kw={'projection': '3d'}, figsize=(8, 6))

for k, lines in layers.items():  
    
#     if filter_layer(k, use_set1=True, 
#                        use_set2=False, 
#                        use_set2_slot=False, 
#                        use_set2_fanned=False):
#         continue
    
    # for the plane work
    xs = []
    ys = []
    zs = []
    
    dataset = [] # for plotting the lines
    
    for e in lines:
        x, y, z = zip(*e.points())
        xs.extend(x)
        ys.extend(y)
        zs.extend(z)
        
        line = {}
        line['x'] = x
        line['y'] = y
        line['z'] = z
        dataset.append(line)

    X = np.array(xs)
    Y = np.array(ys)
    Z = np.array(zs)
    
    print(k) 
    # ===========
    centroid = Point3(np.mean(X), np.mean(Y),np.mean(Z))
    print('Centroid = ({:.4f},{:.4f},{:.4f})'.format(*centroid))
        
    a, b, c, d, plane = best_fit_plane(X, Y, Z)

    # we can't use the d, from the best fit situation, we have to calculate it 
    # relative to the centroid of our data
    d = -centroid.dot(np.array([a, b, c]))
    normal = Vector3(a,b,c)
    print('Plane Normal = ({:.4f},{:.4f},{:.4f})'.format(*normal))
        
    # Aegis requires the azimuth of the ring plane not the normal so subtract
    # 90 degrees to it
    azimuth = np.degrees(np.arccos(normal.dot(Vector3.north_vector())))
    azimuth = np.mod(azimuth - 90.0, 360.0)
    print('Azimuth = {:.4f}'.format(azimuth))
    
    dump = np.degrees(np.arccos(normal.dot(Vector3.up_vector())))
    print('Dump = {:.4f}'.format(dump))    
    # ===========
    
    # ===========
    # setup the grid used to display the best fit plane
    
    data = np.c_[X,Y,Z]
    
    # regular grid covering the domain of the data
    mn = np.min(data, axis=0)
    mx = np.max(data, axis=0)

    if plane == 'XY':
        XX, YY = np.meshgrid(np.linspace(mn[0], mx[0], 5),     
                             np.linspace(mn[1], mx[1], 5))

        ZZ = (-a*XX - b*YY - d)/c

    elif plane == 'XZ':
        XX, ZZ = np.meshgrid(np.linspace(mn[0], mx[0], 5), 
                             np.linspace(mn[2], mx[2], 5))

        YY = (-a*XX - c*ZZ - d)/b

    elif plane == 'YZ':
        YY, ZZ = np.meshgrid(np.linspace(mn[1], mx[1], 5), 
                             np.linspace(mn[2], mx[2], 5))

        XX = (-b*YY - c*ZZ - d)/a
    #==========
    
    cmap = plt.cm.get_cmap('jet')

    rgba = cmap(np.random.normal())
    
    for ds in dataset:
        ax.plot(ds["x"], ds["y"], ds["z"], color=rgba)
       
    ax.plot_wireframe(XX,YY,ZZ, color='k', alpha=0.1)
    
    # ==========
    
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')    

fig.show()       

<IPython.core.display.Javascript object>

95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT
Centroid = (7107.8833,-23230.4002,-994.7378)
Plane Normal = (-0.0405,0.2883,0.9567)
Azimuth = 343.2455
Dump = 16.9237
95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R8A
Centroid = (7111.6793,-23224.1265,-993.0610)
Plane Normal = (-0.9455,0.3256,-0.0000)
Azimuth = 341.0000
Dump = 90.0000
95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R8B
Centroid = (7110.3287,-23223.5054,-993.0939)
Plane Normal = (-0.9925,0.1219,-0.0000)
Azimuth = 353.0000
Dump = 90.0000
95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R8
Centroid = (7109.5299,-23228.1455,-993.1152)
Plane Normal = (0.9984,0.0567,-0.0000)
Azimuth = 356.7481
Dump = 90.0000
95 3W MP2A LH1E_R8-R17_DXF_RINGDESIGN_095 3W MPR02A LHS 01E EXST_SURVEYEXPORT_R9
Centroid = (7107.5078,-23227.8125,-993.2086)
Plane Normal = (0.9984,0.0567,0.0000)
Azimuth = 356.7481
Dump = 90.0000
95 3W

In [35]:
# cyc = plt.cycler(s=np.linspace(200, 50, 3),
#                  cmap=['viridis', 'magma', 'coolwarm'],
#                  alpha=[.25, .5, .75],
#                  lw=[0, .1, .5])

# # You can print the cycler, or use nice jupyter notebook support
# print(cyc)
# dir(cyc)




# Static Data

Gather some more static data for the other notebooks to use

In [36]:
candiate_layers = ['RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R10',
                  'RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R9',
                  'RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R8',
                  'RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R7',
                  'RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R6',
                  'RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R5']

datasets = {}

for layer in candiate_layers:
    
    xs = []
    ys = []
    zs = []
    
    for e in layers[layer]:            
        x, y, z = zip(*e.points())
        xs.extend(x)
        ys.extend(y)
        zs.extend(z)

    xa = np.array(xs)
    ya = np.array(ys)
    za = np.array(zs)
    datasets[layer] = np.c_[xa,ya,za]
        
        
print(datasets)       


KeyError: 'RINGDESIGN_100 04W CUT03 LHP 07W_SURVEYEXPORT_R10'

In [48]:
doc = ezdxf.readfile('dxf/drifts.dxf')

In [44]:
for l in doc.layers:
    print(l)

LAYER(#60340)
LAYER(#60341)
LAYER(#60342)
LAYER(#60343)
LAYER(#60344)
LAYER(#60345)
LAYER(#6036C)


In [54]:
#  iterate over all entities in modelspace
msp = doc.modelspace()
for e in msp:    
    print(f'Layer = {e.dxf.layer}; Type = {e.dxf.dxftype}')    
    break
    
# # entity query for all LINE entities in modelspace
# for e in msp.query('LINE'):
#     print_entity(e)

# def print_entity(e):
#     print("LINE on layer: %s\n" % e.dxf.layer)
#     print("start point: %s\n" % e.dxf.start)
#     print("end point: %s\n" % e.dxf.end)

Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_

Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_

Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_

Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_

Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_25; Type = POLYLINE
Layer = 2_БДШ_

...

Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_

Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_

Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_

Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_

Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_29; Type = POLYLINE
Layer = 2_БДШ_